In [17]:
import pandas as pd
import numpy as np
import json
import os
#collecting contracts name
dataset='smartbugs/data_analysis/'
results_path='../results/'
contracts_path=os.path.join(results_path,dataset,'contracts.csv')
storage_path=os.path.join(results_path,dataset)
patches_path=os.path.join(results_path,dataset,'all_patches_stats.csv')


patches=pd.read_csv(patches_path)


In [18]:
# Generate 10 random samples per source code tool
sofias_tools = [
    'sGuard',
    'sGuardPlus',
    'SmartFix',
    "SolGPT",
    'TIPS',
]

import random as rd
import pandas as pd
import os


for tool in sofias_tools:
    print(tool)
    # Fixed the syntax error: use parentheses around each condition
    patches_tool = patches[(patches['Tool'] == tool) & (patches['mitigates'] == 'yes')]
    
    # Get 10 random samples or all if less than 10 are available
    if len(patches_tool) > 10:
        patches_sample = patches_tool.sample(10, random_state=42)
    else:
        patches_sample = patches_tool
    
    # Save the sampled patches to CSV
    patches_sample.to_csv(os.path.join(storage_path, tool + '_patches.csv'), index=False)
    
    # Print contract names for verification
    print(patches_sample['Patch'].values)
    print('------------------------------------')

sGuard
['mycontract.sol' 'reentrance.sol' 'simple_dao.sol'
 '0x7541b76cb60f4c60af330c208b0623b7f54bf615.sol'
 '0xcead721ef5b11f1a7b530171aab69b16c5e66b6e.sol'
 '0x7a8721a9d64c74da899424c1b52acbf58ddc9782.sol' 'phishable.sol'
 'reentrancy_simple.sol' '0x01f8c4e3fa3edeb29e514cba738d87ce8c091d3f.sol'
 '0xbaf51e761510c1a11bf48dd87c0307ac8a8c8a4f.sol']
------------------------------------
sGuardPlus
['0x7541b76cb60f4c60af330c208b0623b7f54bf615.sol'
 '0xf015c35649c82f5467c9c74b7f28ee67665aad68.sol'
 '0x4e73b32ed6c35f570686b89848e5f39f20ecc106.sol' 'reentrancy_dao.sol'
 '0x7b368c4e805c3870b6c49a3f1f49f69af8662cf3.sol' 'phishable.sol'
 '0x52d2e0f9b01101a59b38a3d05c80b7618aeed984.sol'
 'overflow_simple_add.sol' 'integer_overflow_mul.sol'
 '0xb7c5c5aa4d42967efe906e1b66cb8df9cebf04f7.sol']
------------------------------------
SmartFix
['0x8c7777c45481dba411450c228cb692ac3d550344.sol' 'phishable.sol'
 '0x941d225236464a25eb18076df7da6a91d0f95e9e.sol' 'simple_suicide.sol'
 '0xb93430ce38ac4a6bb47fb1f

In [19]:
vulnerabilities = patches['Category'].unique()

for tool in source_code_tools:
    previous_samples = pd.read_csv(os.path.join(storage_path, tool + '_patches.csv'))


    samples_per_tool = []
    for vuln in vulnerabilities:

        print(f"Tool: {tool}, Vulnerability: {vuln}")
        sample_population = patches[(patches['Tool'] == tool) 
                               & (patches['mitigates'] == 'yes')
                               & (patches['Category'] == vuln)]
        print(f"Sample population size: {len(sample_population)}")
        print(sample_population['Patch'].values)
        
        previous_vuln_samples = previous_samples[previous_samples['Category'] == vuln]
        # Filter out patches that are already in the previous samples per vulnerability
        filtered_patches = sample_population[~sample_population['Patch'].isin(previous_vuln_samples['Patch'])]
        print(filtered_patches['Patch'].values)
        
        patches_sample = previous_vuln_samples

        n = 10 - len(patches_sample)
        print(f"Number of patches to sample: {n}")

        if len(filtered_patches) > n:
            additional_samples = filtered_patches.sample(n, random_state=42)
        else:
            additional_samples = filtered_patches
        # Concatenate the previous samples with the new samples
        patches_sample = pd.concat([patches_sample, additional_samples], ignore_index=True)
        
        print(patches_sample['Patch'].values)
        print('------------------------------------')

        samples_per_tool.append(patches_sample)


    # Concatenate all samples for the current tool
    patches_tool = pd.concat(samples_per_tool, ignore_index=True)
    # Save the sampled patches to CSV
    patches_tool.to_csv(os.path.join(storage_path, tool + '_patches.csv'), index=False)

Tool: sGuard, Vulnerability: access_control
Sample population size: 3
['mycontract.sol' 'phishable.sol' 'wallet_04_confused_sign.sol']
['wallet_04_confused_sign.sol']
Number of patches to sample: 8
['mycontract.sol' 'phishable.sol' 'wallet_04_confused_sign.sol']
------------------------------------
Tool: sGuard, Vulnerability: arithmetic
Sample population size: 2
['tokensalechallenge.sol' 'timelock.sol']
['tokensalechallenge.sol' 'timelock.sol']
Number of patches to sample: 10
['tokensalechallenge.sol' 'timelock.sol']
------------------------------------
Tool: sGuard, Vulnerability: bad_randomness
Sample population size: 0
[]
[]
Number of patches to sample: 10
[]
------------------------------------
Tool: sGuard, Vulnerability: denial_of_service
Sample population size: 0
[]
[]
Number of patches to sample: 10
[]
------------------------------------
Tool: sGuard, Vulnerability: front_running
Sample population size: 0
[]
[]
Number of patches to sample: 10
[]
------------------------------

In [20]:
# verifying that all previous samples are in the new samples

for tool in source_code_tools:
    previous_samples = pd.read_csv(os.path.join(storage_path, tool + '_patches.csv'))
    new_samples = pd.read_csv(os.path.join(storage_path, tool + '_patches_all.csv'))

    # check if all patches in previous samples are in new samples, print the missing ones
    missing_patches = set(previous_samples['Patch']) - set(new_samples['Patch'])
    print(f"Missing patches for {tool}: {missing_patches}")

Missing patches for sGuard: set()
Missing patches for sGuardPlus: set()
Missing patches for SmartFix: set()
Missing patches for SolGPT: set()
Missing patches for TIPS: set()
